In [ ]:
import re
import os
import sys
import time
import itertools
import numpy as np
import scipy as sp
from utils import *
from math import log
from h5py import File
import multiprocessing
from numpy.linalg import *
from scipy.linalg import *
from scipy.io import loadmat
from scipy.optimize import minimize
from mindquantum.framework import *
from mindquantum.core.gates import *
from typing import List, Union, Tuple
from scipy.stats import unitary_group
from mindquantum.core.circuit import *
from IPython.display import display_svg
from mindquantum.core.operators import *
from mindquantum.algorithm.nisq import *
from mindquantum.algorithm.compiler import *
from scipy.sparse import csc_matrix, csr_matrix
from mindquantum.simulator.utils import GradOpsWrapper
from mindquantum.simulator import Simulator, get_supported_simulator

np.set_printoptions(linewidth=1000)

In [ ]:
def Rd(name, ind, pr):
    if len(ind) != 2:
        raise ValueError(f'R{name} index length {len(ind)} should be 2')
    if len(set(ind)) != len(ind):
        raise ValueError(f'R{name} index {ind} cannot be repeated')
    if min(ind) < 0 or max(ind) >= 3:
        raise ValueError(f'R{name} index {ind} should in 0 to 2')
    mat = np.eye(3, dtype=CDTYPE)
    if 'X' in name:
        mat[np.ix_(ind, ind)] = RX(pr).matrix()
    elif 'Y' in name:
        mat[np.ix_(ind, ind)] = RY(pr).matrix()
    elif 'Z' in name:
        mat[np.ix_(ind, ind)] = RZ(pr).matrix()
    return mat


def GCR(name, ind, pr, state, reverse=False):
    if state < 0 or state >= 3:
        raise ValueError(f'¦{state}⟩ control state should in 0 to 2')
    if reverse:
        mat = np.eye(9, dtype=CDTYPE)
        idx = [i for i in range(9) if i % 3 == state]
        mat[np.ix_(idx, idx)] = Rd(name, ind, pr)
    else:
        mat = block_diag(np.eye(3 * state), Rd(name, ind, pr), np.eye(3 * (2 - state)))
    return mat


def Pm(pr, state):
    return block_diag(np.eye(state), np.exp(1j * pr), np.eye(2 - state))


state = 0
pr = [1, 2, 3]
mat = np.kron(Pm(pr[2], state), np.eye(3)) @ GCR('Z', [0, 2], pr[1], state) @ GCR('Z', [0, 1], pr[0], state)
print(mat)

p = np.eye(16) - symmetric_encoding(np.eye(9), 2)
print(symmetric_encoding(mat, 2) + p)

In [ ]:
circ = controlled_diagonal_synthesis(3, 'CD', 0, [1, 2, 3], state)
print(circ.matrix(pr))
# circ.ansatz_params_name
circ.svg()

In [ ]:
mat = np.kron(Pm(pr[2], state), np.eye(3))
print(mat)

p = np.eye(16) - symmetric_encoding(np.eye(9), 2)
print(symmetric_encoding(mat, 2) + p)

In [ ]:
def Pm_synthesis(name: str, ctrl: List[int], state: int) -> Circuit:
    if state == 0:
        corr = Circuit() + X(ctrl[1]) + X(ctrl[2])
    elif state == 1:
        corr = Circuit() + X(ctrl[1], ctrl[2]) + RY(np.pi / 2).on(ctrl[2])
    elif state == 2:
        corr = Circuit()
    circ = Circuit() + corr
    circ += PhaseShift(name).on(ctrl[1], ctrl[2])
    circ += corr.hermitian()
    return circ


circ = Pm_synthesis('Pm', [1, 2, 3], 0)
print(circ.matrix(pr[2]))
circ.svg()

In [ ]:
dim, n_qudits = 3, 1
n_qubits = (dim - 1) * n_qudits
p = np.eye(2**n_qubits) - symmetric_encoding(np.eye(dim), n_qudits)

i, j = 0, 1
mat1 = np.eye(dim, dtype=np.complex128)
mat1[np.ix_([i, j], [i, j])] = np.array([[1, 2], [3, 4]])
mat2 = symmetric_encoding(mat1, n_qudits) + p
print(mat1)
print(mat2)

In [ ]:
def optimization(init_params: np.ndarray, sim_grad: GradOpsWrapper, loss_list: List[float] = None):
    f, g = sim_grad(init_params)
    loss = 1 - np.real(f)[0][0]
    grad = -np.real(g)[0][0]
    if loss_list is not None:
        loss_list.append(loss)
    return loss, grad


for r in range(1, 1000):
    3 = 3  # dimension of qudit state
    p = np.eye(16) - symmetric_encoding(np.eye(9), 2)
    mat = unitary_group.rvs(9)
    mat = symmetric_encoding(mat, 2) + p
    state = np.random.rand(9) + 1j * np.random.rand(9)
    state /= norm(state)

    obj = [0, 1, 2, 3]
    obj = obj[::-1]
    gate = UnivMathGate('mat', mat).on(obj)
    ansatz = qutrit_symmetric_ansatz(gate)

    nq = ansatz.n_qubits
    p_name = ansatz.ansatz_params_name
    p_num = len(p_name)
    g_num = sum(1 for _ in ansatz)
    depth = circuit_depth(ansatz)

    sim_list = set([i[0] for i in get_supported_simulator()])
    if 'mqvector_gpu' in sim_list and nq > 14:
        sim = Simulator('mqvector_gpu', nq)
        method = 'BFGS'
    else:
        sim = Simulator('mqvector', nq)
        method = 'BFGS'  # TNC CG

    psi = symmetric_encoding(state, 2, is_csr=True)  # encode qutrit state to qubit
    rho = psi.dot(psi.conj().T)  # rho & psi are both csr_matrix
    Ham = Hamiltonian(rho)  # set target state as Hamiltonian

    start = time.perf_counter()
    sim.reset()  # reset simulator to zero state
    sim_grad = sim.get_expectation_with_grad(Ham, ansatz)
    while True:
        try:
            local_minima1, local_minima2 = [], []
            solver_options = {'gtol': 1e-20, 'maxiter': 1000}
            init_params = np.random.uniform(-np.pi, np.pi, p_num)
            res = minimize(optimization, init_params, (sim_grad, []), method, jac=True, options=solver_options)
            break
        except StopIteration:
            break  # reach loss tolerance
        except StopAsyncIteration:
            continue  # reach local minima
    end = time.perf_counter()
    seconds = round(end - start, 2)
    sim.reset()  # reset simulator to zero state
    pr_res = dict(zip(p_name, res.x))  # optimal result parameters
    sim.apply_circuit(ansatz.apply_value(pr_res))  # apply result params to circuit
    psi_res = sim.get_qs()  # get result pure state

    if not np.allclose(fidelity(psi_res, psi.toarray()), 1):
        print(f'{res.message}\nOptimal: {res.fun}, Fidelity: {1-res.fun:.20f} {res.nfev} {seconds}')
        print(fidelity(psi_res, psi.toarray()))

In [ ]:
def Rd(d, name, ind, pr):
    if len(ind) != 2:
        raise ValueError(f'R{name} index length {len(ind)} should be 2')
    if len(set(ind)) != len(ind):
        raise ValueError(f'R{name} index {ind} cannot be repeated')
    if min(ind) < 0 or max(ind) >= d:
        raise ValueError(f'R{name} index {ind} should in 0 to {d-1}')
    mat = np.eye(d, dtype=CDTYPE)
    if 'RX' in name:
        mat[np.ix_(ind, ind)] = RX(pr).matrix()
    elif 'RY' in name:
        mat[np.ix_(ind, ind)] = RY(pr).matrix()
    elif 'RZ' in name:
        mat[np.ix_(ind, ind)] = RZ(pr).matrix()
    return mat


def GCR(d, name, ind, pr, state, reverse=False):
    if state < 0 or state >= d:
        raise ValueError(f'¦{state}⟩ control state should in 0 to {d-1}')
    if reverse:
        mat = np.eye(d**2, dtype=CDTYPE)
        idx = [i for i in range(d**2) if i % d == state]
        mat[np.ix_(idx, idx)] = Rd(d, name, ind, pr)
    else:
        mat = block_diag(np.eye(d * state), Rd(d, name, ind, pr), np.eye(d * (d - state - 1)))
    return mat


def Sm(d, pr, state):
    return block_diag(np.eye(state), np.exp(1j * pr), np.eye(d - 1 - state))


def Sm_synthesis(d: int, name: str, obj: int, ctrl: List[int], state: int) -> Circuit:
    if d != 3:
        raise ValueError('Only works when d = 3')
    if state == 0:
        corr = Circuit() + X(ctrl[1]) + X(ctrl[2])
    elif state == 1:
        corr = Circuit() + X(ctrl[2], ctrl[1]) + RY(np.pi / 2).on(ctrl[1])
    elif state == 2:
        corr = Circuit()
    circ = Circuit() + corr
    circ += PhaseShift(name).on(ctrl[2], ctrl[1])
    circ += corr.hermitian()
    return circ


def controlled_diagonal_synthesis(d: int, name: str, obj: int, ctrl: List[int], state: int) -> Circuit:
    if d != 3:
        raise ValueError('Only works when d = 3')
    if state == 0:
        corr = Circuit() + X(ctrl[1]) + X(ctrl[2])
    elif state == 1:
        corr = Circuit() + X(ctrl[1], ctrl[2]) + RY(np.pi / 2).on(ctrl[2])
    elif state == 2:
        corr = Circuit()
    ind01 = Circuit() + X(obj, ctrl) + RY(-np.pi / 2).on(ctrl[0], [obj] + ctrl[1:]) + X(ctrl[0], ctrl[1:])
    ind02 = Circuit() + X(ctrl[0], ctrl[1:] + [obj]) + X(ctrl[0], ctrl[1:])
    ind12 = Circuit() + X(obj, ctrl) + RY(np.pi / 2).on(ctrl[0], [obj] + ctrl[1:]) + X(obj, ctrl[1:])
    circ = Circuit() + corr
    circ = circ + ind01 + RZ(f'{name}RZ01').on(obj, ctrl) + ind01.hermitian()
    circ = circ + ind02 + RZ(f'{name}RZ02').on(obj, ctrl) + ind02.hermitian()
    circ += PhaseShift(name).on(ctrl[2], ctrl[1])
    # circ = circ + ind01 + GlobalPhase(f'{name}GP').on(obj, ctrl) + ind01.hermitian()
    # circ = circ + ind02 + GlobalPhase(f'{name}GP').on(obj, ctrl) + ind02.hermitian()
    # circ = circ + ind12 + GlobalPhase(f'{name}GP').on(obj, ctrl) + ind12.hermitian()
    circ += corr.hermitian()
    return circ

In [ ]:
for i in range(1000):
    3, obj, ctrl, state = 3, 0, [1, 2, 3], i % 3
    pr = np.random.rand(3)
    circ = controlled_diagonal_synthesis(3, '', obj, ctrl, state)
    nq = circ.n_qubits

    p = np.eye(2**((3 - 1) * 2)) - symmetric_encoding(np.eye(3**2), 2)
    p1 = np.eye(2**(3 - 1)) - symmetric_encoding(np.eye(3))

    # CD = GCR(d, 'RZ02', [0, 2], pr[1], state) @ GCR(d, 'RZ01', [0, 1], pr[0], state)
    # CD = symmetric_encoding(CD, 2) + p
    # CD = np.kron(symmetric_encoding(Sm(d, pr[2], state)) + p1, np.eye(4)) @ CD

    CD = np.kron(Sm(3, pr[2], state), np.eye(3)) @ GCR(3, 'RZ02', [0, 2], pr[1], state) @ GCR(3, 'RZ01', [0, 1], pr[0], state)
    CD = symmetric_encoding(CD, 2) + p

    a1 = np.random.rand(3) + 1j * np.random.rand(3)
    a2 = np.random.rand(3) + 1j * np.random.rand(3)
    a1 /= norm(a1)
    a2 /= norm(a2)
    b1 = np.kron(a1, a2)
    b2 = symmetric_encoding(b1, 2)

    sim = Simulator('mqvector', nq)
    sim.set_qs(b2)
    sim.apply_circuit(circ, pr)
    if not np.allclose(sim.get_qs(), CD @ b2):
        print(sim.get_qs())
        print(CD @ b2)
        print(approx_matrix(sim.get_qs() - CD @ b2))
circ.svg()

In [ ]:
def Pd(name, d, ind):
    if len(ind) != 2:
        raise ValueError(f'{name} index length {len(ind)} should be 2')
    if len(set(ind)) != len(ind):
        raise ValueError(f'{name} index {ind} cannot be repeated')
    if min(ind) < 0 or max(ind) >= d:
        raise ValueError(f'{name} index {ind} should in 0 to {d-1}')
    mat = np.eye(d, dtype=CDTYPE)
    if name == 'X':
        mat[np.ix_(ind, ind)] = X.matrix()
    elif name == 'Y':
        mat[np.ix_(ind, ind)] = Y.matrix()
    elif name == 'Z':
        mat[np.ix_(ind, ind)] = Z.matrix()
    return mat


def Rd(name, d, ind, pr):
    if len(ind) != 2:
        raise ValueError(f'R{name} index length {len(ind)} should be 2')
    if len(set(ind)) != len(ind):
        raise ValueError(f'R{name} index {ind} cannot be repeated')
    if min(ind) < 0 or max(ind) >= d:
        raise ValueError(f'R{name} index {ind} should in 0 to {d-1}')
    mat = np.eye(d, dtype=CDTYPE)
    if name == 'X':
        mat[np.ix_(ind, ind)] = RX(pr).matrix()
    elif name == 'Y':
        mat[np.ix_(ind, ind)] = RY(pr).matrix()
    elif name == 'Z':
        mat[np.ix_(ind, ind)] = RZ(pr).matrix()
    return mat


def Ud(d, ind, pr):
    if len(ind) != 2:
        raise ValueError(f'U3 index length {len(ind)} should be 2')
    if len(set(ind)) != len(ind):
        raise ValueError(f'U3 index {ind} cannot be repeated')
    if min(ind) < 0 or max(ind) >= d:
        raise ValueError(f'U3 index {ind} should in 0 to {d-1}')
    if len(pr) != 3:
        raise ValueError(f'U3 params length {len(pr)} should be 3')
    theta, phi, lam = pr
    mat = np.eye(d, dtype=CDTYPE)
    mat[np.ix_(ind, ind)] = U3(theta, phi, lam).matrix()
    return mat


def INC(d):
    mat = np.eye(d, dtype=CDTYPE)
    mat = mat[[d - 1] + list(range(d - 1))]
    return mat


def CINC(d):
    return block_diag(np.eye(d**2 - d), INC(d))


def GCX(d, ind, m):
    if m < 0 or m >= d:
        raise ValueError(f'¦{m}⟩ control state should in 0 to {d-1}')
    return block_diag(np.eye(d * m), Pd('X', d, ind), np.eye(d * (d - m - 1)))

def GCP(name, d, ind, m):
    if m < 0 or m >= d:
        raise ValueError(f'¦{m}⟩ control state should in 0 to {d-1}')
    return block_diag(np.eye(d * m), Pd(name, d, ind), np.eye(d * (d - m - 1)))

def GCR(name, d, ind, pr, m):
    if m < 0 or m >= d:
        raise ValueError(f'¦{m}⟩ control state should in 0 to {d-1}')
    return block_diag(np.eye(d * m), Rd(name, d, ind, pr), np.eye(d * (d - m - 1)))

def GCU(d, ind, U, m):
    if m < 0 or m >= d:
        raise ValueError(f'¦{m}⟩ control state should in 0 to {d-1}')
    mat = np.eye(d, dtype=CDTYPE)
    mat[np.ix_(ind, ind)] = U
    return block_diag(np.eye(d * m), mat, np.eye(d * (d - m - 1)))

In [ ]:
def fun(p0, sim_grad, args=None):
    f, g = sim_grad(p0)
    f = 1 - np.real(f)[0][0]
    g = -np.real(g)[0][0]
    if args is not None:
        args.append(f)
        i = len(args)
        if i % 10 == 0:
            global start
            t = time.perf_counter() - start
            print('Loss: %.15f, Fidelity: %.15f, %3d, %.4f' % (f, 1 - f, i, t))
    return f, g


def Uind(basis, d, ind, pr, obj):
    if d != 3:
        raise ValueError('Only works when d = 3')
    if len(ind) != 2:
        raise ValueError(f'U3 index length {len(ind)} should be 2')
    if len(set(ind)) != len(ind):
        raise ValueError(f'U3 index {ind} cannot be repeated')
    if min(ind) < 0 or max(ind) >= d:
        raise ValueError(f'U3 index {ind} should in 0 to {d-1}')
    if len(pr) != 3:
        raise ValueError(f'U3 params length {len(pr)} should be 3')
    circ = Circuit()
    if ind == [0, 1]:
        corr = Circuit() + X(obj[1], obj[0]) + RY(np.pi / 2).on(obj[0], obj[1]) + X(obj[1], obj[0]) + X(obj[1])
    elif ind == [0, 2]:
        corr = Circuit() + X(obj[0]) + X(obj[1], obj[0]) + X(obj[0])
    elif ind == [1, 2]:
        corr = Circuit() + X(obj[1], obj[0]) + RY(-np.pi / 2).on(obj[0], obj[1]) + X(obj[1], obj[0])
    circ += corr
    if basis == 'zyz':
        circ += RZ(pr[0]).on(obj[0], obj[1])
        circ += RY(pr[1]).on(obj[0], obj[1])
        circ += RZ(pr[2]).on(obj[0], obj[1])
    elif basis == 'u3':
        theta, phi, lam = pr
        circ += U3(theta, phi, lam).on(obj[0], obj[1])
    else:
        raise ValueError(f'Wrong basis {basis} is not in {opt_basis}')
    circ += corr.hermitian()
    return circ


def Ub(basis, d, name, obj):
    circ = Circuit()
    index = [[0, 2], [1, 2], [0, 2]]
    if basis == 'zyz':
        for i, ind in enumerate(index):
            str_pr = f'{"".join(str(i) for i in ind)}_{i}'
            pr = [f'{name}RZ{str_pr}', f'{name}RY{str_pr}', f'{name}Rz{str_pr}']
            circ += Uind(basis, d, ind, pr, obj)
    elif basis == 'u3':
        for i, ind in enumerate(index):
            str_pr = f'{"".join(str(i) for i in ind)}_{i}'
            pr = [f'{name}𝜃{str_pr}', f'{name}𝜑{str_pr}', f'{name}𝜆{str_pr}']
            circ += Uind(basis, d, ind, pr, obj)
    else:
        raise ValueError(f'Wrong basis {basis} is not in {opt_basis}')
    return circ


def GCRb(d, ind, name, obj, ctrl, state):
    if d != 3:
        raise ValueError('Only works when d = 3')
    circ = Circuit()
    if state == 0:
        if ind == [0, 1]:
            corr = Circuit() + X(ctrl[1]) + X(ctrl[2]) + X(ctrl[0], ctrl[1:] + [obj]) + RY(np.pi / 2).on(obj, ctrl) + X(
                ctrl[0], ctrl[1:] + [obj]) + X(ctrl[0], ctrl[1:])
        elif ind == [0, 2]:
            corr = Circuit() + X(ctrl[1]) + X(ctrl[2]) + X(obj, ctrl[1:]) + X(ctrl[0], ctrl[1:] + [obj]) + X(
                obj, ctrl[1:])
        elif ind == [1, 2]:
            corr = Circuit() + X(ctrl[1]) + X(ctrl[2]) + X(ctrl[0], ctrl[1:] + [obj]) + RY(-np.pi / 2).on(
                obj, ctrl) + X(ctrl[0], ctrl[1:] + [obj])
    elif state == 1:
        if ind == [0, 1]:
            corr = Circuit() + X(ctrl[1], ctrl[2]) + RY(np.pi / 2).on(ctrl[2]) + X(ctrl[0], ctrl[1:] + [obj]) + RY(
                np.pi / 2).on(obj, ctrl) + X(ctrl[0], ctrl[1:] + [obj]) + X(ctrl[0], ctrl[1:])
        elif ind == [0, 2]:
            corr = Circuit() + X(ctrl[1], ctrl[2]) + RY(np.pi / 2).on(ctrl[2]) + X(obj, ctrl[1:]) + X(
                ctrl[0], ctrl[1:] + [obj]) + X(obj, ctrl[1:])
        elif ind == [1, 2]:
            corr = Circuit() + X(ctrl[1], ctrl[2]) + RY(np.pi / 2).on(ctrl[2]) + X(ctrl[0], ctrl[1:] + [obj]) + RY(
                -np.pi / 2).on(obj, ctrl) + X(ctrl[0], ctrl[1:] + [obj])
    elif state == 2:
        if ind == [0, 1]:
            corr = Circuit() + X(ctrl[0], ctrl[1:] + [obj]) + RY(np.pi / 2).on(obj, ctrl) + X(
                ctrl[0], ctrl[1:] + [obj]) + X(ctrl[0], ctrl[1:])
        elif ind == [0, 2]:
            corr = Circuit() + X(obj, ctrl[1:]) + X(ctrl[0], ctrl[1:] + [obj]) + X(obj, ctrl[1:])
        elif ind == [1, 2]:
            corr = Circuit() + X(ctrl[0], ctrl[1:] + [obj]) + RY(-np.pi / 2).on(obj, ctrl) + X(
                ctrl[0], ctrl[1:] + [obj])
    circ += corr
    if 'RX' in name:
        circ = circ + RX(name).on(obj, ctrl)
    elif 'RY' in name:
        circ = circ + RY(name).on(obj, ctrl)
    elif 'RZ' in name:
        circ = circ + RZ(name).on(obj, ctrl)
    elif 'GP' in name:
        circ = circ + GlobalPhase(name).on(obj, ctrl)
    circ += corr.hermitian()
    return circ


def GCPb(d, name, obj, ctrl, state):
    if d != 3:
        raise ValueError('Only works when d = 3')
    circ = Circuit()
    if state == 0:
        corr = Circuit()
    elif state == 1:
        corr = Circuit()
    elif state == 2:
        corr = Circuit()
    circ += corr
    circ = circ + GlobalPhase(name).on(obj, ctrl)
    circ += corr.hermitian()
    return circ


def Cb(d, name, obj, ctrl, state):
    if d != 3:
        raise ValueError('Only works when d = 3')
    circ = Circuit()
    circ += GCRb(d, [0, 1], f'{name}RZ01', obj, ctrl, state)
    circ += GCRb(d, [0, 2], f'{name}RZ02', obj, ctrl, state)
    circ += GCRb(d, [0, 1], f'{name}GP', obj, ctrl, state)
    circ += GCRb(d, [0, 2], f'{name}GP', obj, ctrl, state)
    circ += GCRb(d, [1, 2], f'{name}GP', obj, ctrl, state)
    return circ


def qutrit_symmetric_ansatz(gate: UnivMathGate, basis: str = 'zyz', with_phase: bool = False):
    name = f'{gate.name}_'
    obj = gate.obj_qubits
    circ = Circuit()
    if len(obj) == 2:
        circ += Ub(basis, 3, f'{name}', obj)
    elif len(obj) == 4:
        circ += Ub(basis, 3, f'{name}U1_', obj[:2])
        circ += Cb(3, f'{name}C1_', obj[0], obj[1:], 1)
        circ += Ub(basis, 3, f'{name}U2_', obj[:2])
        circ += Cb(3, f'{name}C2_', obj[0], obj[1:], 2)
        circ += Ub(basis, 3, f'{name}U3_', obj[:2])
        circ += GCRb(3, [1, 2], f'{name}RY12', obj[-1], obj[::-1][1:], 2)
        circ += GCRb(3, [1, 2], f'{name}RY11', obj[-1], obj[::-1][1:], 1)
        circ += GCRb(3, [1, 2], f'{name}RY10', obj[-1], obj[::-1][1:], 0)
        circ += Ub(basis, 3, f'{name}U4_', obj[:2])
        circ += Cb(3, f'{name}C3_', obj[0], obj[1:], 2)
        circ += Ub(basis, 3, f'{name}U5_', obj[:2])
        circ += GCRb(3, [0, 1], f'{name}RY22', obj[-1], obj[::-1][1:], 2)
        circ += GCRb(3, [0, 1], f'{name}RY21', obj[-1], obj[::-1][1:], 1)
        circ += GCRb(3, [0, 1], f'{name}RY20', obj[-1], obj[::-1][1:], 0)
        circ += Ub(basis, 3, f'{name}U6_', obj[:2])
        circ += Cb(3, f'{name}C4_', obj[0], obj[1:], 0)
        circ += Ub(basis, 3, f'{name}U7_', obj[:2])
        circ += GCRb(3, [1, 2], f'{name}RY32', obj[-1], obj[::-1][1:], 2)
        circ += GCRb(3, [1, 2], f'{name}RY31', obj[-1], obj[::-1][1:], 1)
        circ += GCRb(3, [1, 2], f'{name}RY30', obj[-1], obj[::-1][1:], 0)
        circ += Ub(basis, 3, f'{name}U8_', obj[:2])
        circ += Cb(3, f'{name}C5_', obj[0], obj[1:], 2)
        circ += Ub(basis, 3, f'{name}U9_', obj[:2])
    else:
        raise ValueError('Only works when number of qutrits <= 2')
    if with_phase:
        for i in obj:
            circ += GlobalPhase(f'{name}phase').on(i)
    return circ


opt_basis = ['zyz', 'u3']

3, state = 3, 2
nqd = (3 - 1) * state
Id = symmetric_encoding(np.eye(3**state), state)
p = np.eye(Id.shape[0]) - Id

circ = Circuit()
ansatz = Circuit()
mat = unitary_group.rvs(3**state)
mat = symmetric_encoding(mat, state) + p
gate = UnivMathGate(f'mat', mat).on(list(range(nqd)))
circ += gate
ansatz += qutrit_symmetric_ansatz(gate, 'zyz')

nqd = circ.n_qubits
sim = Simulator('mqvector', nqd)
sim.apply_circuit(circ)
psi = sim.get_qs()

rho = np.outer(psi, psi.conj())
Ham = Hamiltonian(csr_matrix(rho))
print('Hamiltonian Dimension:', rho.shape)

p_name = ansatz.ansatz_params_name
p_num = len(p_name)
g_num = sum(1 for _ in ansatz)
print('Number of qubits: %d' % nqd)
print('Number of params: %d' % p_num)
print('Number of gates: %d' % g_num)

sim.reset()
sim_grad = sim.get_expectation_with_grad(Ham, ansatz)
p0 = np.random.uniform(-1, 1, p_num)
start = time.perf_counter()
fun(p0, sim_grad)
res = minimize(fun, p0, args=(sim_grad, []), method='CG', jac=True, tol=1e-8)

print(res.message)
print('Optimal: %.20f' % res.fun)

sim.reset()
pr_res = dict(zip(p_name, res.x))
sim.apply_circuit(ansatz.apply_value(pr_res))
psi_res = sim.get_qs()
print('psi norm: %.20f' % norm(psi - psi_res, 2))
print('psi fidelity: %.20f' % fidelity(psi, psi_res))
print('Is state symmetric:', is_symmetric(psi, state), is_symmetric(psi_res, state))

ansatz.svg()

In [ ]:
def fun(p0, sim_grad, args=None):
    f, g = sim_grad(p0)
    f = 1 - np.real(f)[0][0]
    g = -np.real(g)[0][0]
    if args is not None:
        args.append(f)
        i = len(args)
        if i % 10 == 0:
            global start
            t = time.perf_counter() - start
            print('Loss: %.15f, Fidelity: %.15f, %3d, %.4f' % (f, 1 - f, i, t))
    return f, g


def Ub(basis, d, ind, pr, obj):
    if d != 3:
        raise ValueError('Only works when d = 3')
    if len(ind) != 2:
        raise ValueError(f'U3 index length {len(ind)} should be 2')
    if len(set(ind)) != len(ind):
        raise ValueError(f'U3 index {ind} cannot be repeated')
    if min(ind) < 0 or max(ind) >= d:
        raise ValueError(f'U3 index {ind} should in 0 to {d-1}')
    if len(pr) != 3:
        raise ValueError(f'U3 params length {len(pr)} should be 3')
    circ = Circuit()
    if ind == [0, 1]:
        corr = Circuit() + X(obj[1], obj[0]) + RY(np.pi / 2).on(obj[0], obj[1]) + X(obj[1], obj[0]) + X(obj[1])
    elif ind == [0, 2]:
        corr = Circuit() + X(obj[0]) + X(obj[1], obj[0]) + X(obj[0])
    elif ind == [1, 2]:
        corr = Circuit() + X(obj[1], obj[0]) + RY(-np.pi / 2).on(obj[0], obj[1]) + X(obj[1], obj[0])
    circ += corr
    if basis == 'zyz':
        circ += RZ(pr[0]).on(obj[0], obj[1])
        circ += RY(pr[1]).on(obj[0], obj[1])
        circ += RZ(pr[2]).on(obj[0], obj[1])
    elif basis == 'u3':
        theta, phi, lam = pr
        circ += U3(theta, phi, lam).on(obj[0], obj[1])
    else:
        raise ValueError(f'Wrong basis {basis} is not in {opt_basis}')
    circ += corr.hermitian()
    return circ


def one_qutrit_symmetric_ansatz(gate: UnivMathGate, basis: str = 'zyz', with_phase: bool = False):
    name = f'{gate.name}_'
    obj = gate.obj_qubits
    circ = Circuit()
    index = [[0, 2], [1, 2], [0, 2]]
    if basis == 'zyz':
        for i, ind in enumerate(index):
            str_pr = f'{"".join(str(i) for i in ind)}_{i}'
            pr = [f'{name}RZ{str_pr}', f'{name}RY{str_pr}', f'{name}Rz{str_pr}']
            circ += Ub(basis, 3, ind, pr, obj)
    elif basis == 'u3':
        for i, ind in enumerate(index):
            str_pr = f'{"".join(str(i) for i in ind)}_{i}'
            pr = [f'{name}𝜃{str_pr}', f'{name}𝜑{str_pr}', f'{name}𝜆{str_pr}']
            circ += Ub(basis, 3, ind, pr, obj)
    else:
        raise ValueError(f'Wrong basis {basis} is not in {opt_basis}')
    if with_phase:
        for i in obj:
            circ += GlobalPhase(f'phase').on(i)
    return circ


opt_basis = ['zyz', 'u3']

3, state = 3, 2
nqd = 3 - 1
Id = symmetric_encoding(np.eye(3))
p = np.eye(Id.shape[0]) - Id

mat = {}
circ = Circuit()
ansatz = Circuit()
for i in range(state):
    mat[i] = unitary_group.rvs(3)
    mat[i + state] = symmetric_encoding(mat[i]) + p
    ctrl = list(range(nqd * i, nqd * (i + 1)))
    gate = UnivMathGate(f'mat{i}', mat[i + state]).on(ctrl)
    circ += gate
    ansatz += one_qutrit_symmetric_ansatz(gate, 'zyz')

nqd = circ.n_qubits
sim = Simulator('mqvector', nqd)
sim.apply_circuit(circ)
psi = sim.get_qs()

rho = np.outer(psi, psi.conj())
Ham = Hamiltonian(csr_matrix(rho))
print('Hamiltonian Dimension:', rho.shape)

p_name = ansatz.ansatz_params_name
p_num = len(p_name)
g_num = sum(1 for _ in ansatz)
print('Number of qubits: %d' % nqd)
print('Number of params: %d' % p_num)
print('Number of gates: %d' % g_num)

sim.reset()
sim_grad = sim.get_expectation_with_grad(Ham, ansatz)
p0 = np.random.uniform(-1, 1, p_num)
fun(p0, sim_grad)
start = time.perf_counter()
res = minimize(fun, p0, args=(sim_grad, []), method='CG', jac=True, tol=1e-8)

print(res.message)
print('Optimal: %.20f' % res.fun)

sim.reset()
pr_res = dict(zip(p_name, res.x))
sim.apply_circuit(ansatz.apply_value(pr_res))
psi_res = sim.get_qs()
print('psi norm: %.20f' % norm(psi - psi_res, 2))
print('psi fidelity: %.20f' % fidelity(psi, psi_res))
print('Is state symmetric:', is_symmetric(psi, state), is_symmetric(psi_res, state))

ansatz.svg()

In [ ]:
# Qubit ansatz of qutrit controlled unitary gate
3 = 3
state = 1
ind = [0, 1]
U = unitary_group.rvs(2)
print(GCU(3, ind, U, state))

p = np.eye(2**((3 - 1) * 2)) - symmetric_encoding(np.eye(3**2), 2)
GCUe = symmetric_encoding(GCU(3, ind, U, state), 2)
GCUp = GCUe + p
# print(GCUp)

circ = Circuit()
if state == 0:
    if ind == [0, 1]:
        corr = Circuit() + X(2) + X(3) + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(
            1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(2) + X(3) + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(2) + X(3) + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
elif state == 1:
    if ind == [0, 1]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
elif state == 2:
    if ind == [0, 1]:
        corr = Circuit() + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
circ += corr
circ += UnivMathGate('U', U).on(0, [1, 2, 3])
circ += corr.hermitian()
# print(circ.matrix())
print(symmetric_decoding(circ.matrix() - p, 2))
print(np.allclose(GCUp, circ.matrix()))
circ.svg()

In [ ]:
# Qubit ansatz of qutrit controlled rotation gate
3 = 3
state = 1
name = 'X'
ind = [0, 1]
t = np.pi / 3
# print(GCR(name, d, ind, t, m))

p = np.eye(2**((3 - 1) * 2)) - symmetric_encoding(np.eye(3**2), 2)
GCRe = symmetric_encoding(GCR(name, 3, ind, t, state), 2)
GCRp = GCRe + p
# print(GCRp)

circ = Circuit()
if state == 0:
    if ind == [0, 1]:
        corr = Circuit() + X(2) + X(3) + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(
            1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(2) + X(3) + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(2) + X(3) + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
elif state == 1:
    if ind == [0, 1]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
elif state == 2:
    if ind == [0, 1]:
        corr = Circuit() + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
circ += corr
if name == 'X':
    circ = circ + RX(t).on(0, [1, 2, 3])
elif name == 'Y':
    circ = circ + RY(t).on(0, [1, 2, 3])
elif name == 'Z':
    circ = circ + RZ(t).on(0, [1, 2, 3])
circ += corr.hermitian()
# print(circ.matrix())
print(symmetric_decoding(circ.matrix() - p, 2))
print(np.allclose(GCRp, circ.matrix()))
circ.svg()

In [ ]:
# Qubit ansatz of qutrit controlled Pauli gate
3 = 3
state = 1
name = 'Y'
ind = [0, 1]
print(GCP(name, 3, ind, state))

p = np.eye(2**((3 - 1) * 2)) - symmetric_encoding(np.eye(3**2), 2)
GCPe = symmetric_encoding(GCP(name, 3, ind, state), 2)
GCPp = GCPe + p
# print(GCPp)

circ = Circuit()
if state == 0:
    if ind == [0, 1]:
        corr = Circuit() + X(2) + X(3) + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(2) + X(3) + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(2) + X(3) + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
elif state == 1:
    if ind == [0, 1]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
elif state == 2:
    if ind == [0, 1]:
        corr = Circuit() + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(0, [2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
circ += corr
if name == 'X':
    circ = circ + X(0, [1, 2, 3])
elif name == 'Y':
    circ = circ + Y(0, [1, 2, 3])
elif name == 'Z':
    circ = circ + Z(0, [1, 2, 3])
circ += corr.hermitian()
# print(circ.matrix())
print(symmetric_decoding(circ.matrix() - p, 2))
print(np.allclose(GCPp, circ.matrix()))
circ.svg()

In [ ]:
# Qubit ansatz of qutrit GCX gate
3 = 3
state = 1
ind = [0, 1]
print(GCX(3, ind, state).real)

p = np.eye(2**((3 - 1) * 2)) - symmetric_encoding(np.eye(3**2), 2)
GCXe = symmetric_encoding(GCX(3, ind, state), 2)
GCXp = GCXe + p
# print(GCXp.real)

circ = Circuit()
if state == 0:
    if ind == [0, 1]:
        corr = Circuit() + X(2) + X(3) + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(2) + X(3) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(2) + X(3) + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3])
elif state == 1:
    if ind == [0, 1]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(2, 3) + RY(np.pi / 2).on(3) + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3])
elif state == 2:
    if ind == [0, 1]:
        corr = Circuit() + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [2, 3])
    elif ind == [0, 2]:
        corr = Circuit() + X(0, [2, 3])
    elif ind == [1, 2]:
        corr = Circuit() + X(1, [0, 2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3])
circ += corr
circ = circ + SWAP([0, 1], [2, 3])
circ += corr.hermitian()
print(circ.matrix().real)
print(symmetric_decoding(circ.matrix() - p, 2).real)
print(np.allclose(GCXp, circ.matrix()))
circ.svg()

In [ ]:
# Qubit ansatz of one-qutrit unitary gate
3 = 3
ind = [0, 2]
U = unitary_group.rvs(2)
Ud = np.eye(3, dtype=CDTYPE)
Ud[np.ix_(ind, ind)] = U
print(Ud)

p = np.eye(2**(3 - 1)) - symmetric_encoding(np.eye(3))
Ue = symmetric_encoding(Ud)
Up = Ue + p
print(Up)

circ = Circuit()
if ind == [0, 1]:
    corr = Circuit() + X(1, 0) + RY(np.pi / 2).on(0, 1) + X(1, 0) + X(1)
    corr = Circuit() + X(0, 1) + X(1) + RY(np.pi / 2).on(1, 0)
    corr = Circuit() + X(0, 1) + RY(-np.pi / 2).on(1, 0) + X(1)
elif ind == [0, 2]:
    corr = Circuit() + X(0) + X(1, 0) + X(0)
    corr = Circuit() + X(1) + X(1, 0)
    corr = Circuit() + X(1, 0) + X(1)
elif ind == [1, 2]:
    corr = Circuit() + X(1, 0) + RY(-np.pi / 2).on(0, 1) + X(1, 0)
    corr = Circuit() + X(0, 1) + RY(np.pi / 2).on(1, 0) + X(0, 1)
    corr = Circuit() + X(0, 1) + RY(np.pi / 2).on(1, 0) + X(0)
circ += corr
circ += UnivMathGate('U', U).on(0, 1)
circ += corr.hermitian()
print(circ.matrix())
# print(symmetric_decoding(circ.matrix() - p))
print(np.allclose(Up, circ.matrix()))
circ.svg()

In [ ]:
# Qubit ansatz of one-qutrit unitary gate
3 = 3
name = 'X'
ind = [0, 1]
pr = np.pi / 3
print(Rd(name, 3, ind, pr))

p = np.eye(2**(3 - 1)) - symmetric_encoding(np.eye(3))
Re = symmetric_encoding(Rd(name, 3, ind, pr))
Rp = Re + p
print(Rp)

circ = Circuit()
if ind == [0, 1]:
    corr = Circuit() + X(1, 0) + RY(np.pi / 2).on(0, 1) + X(1, 0) + X(1)
elif ind == [0, 2]:
    corr = Circuit() + X(0) + X(1, 0) + X(0)
elif ind == [1, 2]:
    corr = Circuit() + X(1, 0) + RY(-np.pi / 2).on(0, 1) + X(1, 0)
circ += corr
if name == 'X':
    circ = circ + RX(pr).on(0, 1)
elif name == 'Y':
    circ = circ + RY(pr).on(0, 1)
elif name == 'Z':
    circ = circ + RZ(pr).on(0, 1)
circ += corr.hermitian()
print(circ.matrix())
print(symmetric_decoding(circ.matrix() - p))
print(np.allclose(Rp, circ.matrix()))
circ.svg()

In [ ]:
# Qubit ansatz of qutrit INC gate
3 = 3
p = np.eye(2**(3 - 1)) - symmetric_encoding(np.eye(3))
q = {i: np.eye(3)[i] for i in range(3)}

state = q[0] + q[1]
state /= norm(state)
# str_ket(state, d)
# str_ket(INC @ state, d)
print(INC(3))

INCe = symmetric_encoding(INC(3))
INCp = INCe + p
# print(INCe)
print(INCp)
# str_ket(symmetric_encoding(state))
# str_ket(INCp @ symmetric_encoding(state))

circ = Circuit() + X(1, 0) + RY(np.pi / 2).on(0, 1) + X(1, 0) + X(0) + SWAP([0, 1]) + X(0) + RY(-np.pi / 2).on(0, 1) + X(1, 0)
print(circ.matrix())
print(symmetric_decoding(circ.matrix() - p))
print(np.allclose(INCp, circ.matrix()))
circ.svg()

In [ ]:
# Qubit ansatz of qutrit CINC gate
3 = 3
print(CINC(3).real)

p = np.eye(2**((3 - 1) * 2)) - symmetric_encoding(np.eye(3**2), 2)
CINCe = symmetric_encoding(CINC(3), 2)
CINCp = CINCe + p
# print(CINCp.real)
circ = Circuit() + X(1, [0, 2, 3]) + RY(np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3]) + X(0, [2, 3])
circ += SWAP([0, 1], [2, 3])
circ = circ + X(0, [2, 3]) + RY(-np.pi / 2).on(0, [1, 2, 3]) + X(1, [0, 2, 3])
print(circ.matrix().real)
# print(symmetric_decoding(circ.matrix() - p, 2))
print(np.allclose(CINCp, circ.matrix()))
circ.svg()

In [ ]:
3, state = 3, 2
nqd = (3 - 1) * state
n = 2**nqd
symmetric_index(3, 2)

ind_ = {}
for i in range(2**(3 - 1)):
    num1 = bin(i).count('1')
    i_ = bin(i)[2:].zfill(3 - 1)
    if num1 in ind_:
        ind_[num1].append(i_)
    else:
        ind_[num1] = [i_]
for i in range(3**state):
    multi = ['']
    base = np.base_repr(i, 3).zfill(state)
    for j in range(state):
        multi = [x + y for x in multi for y in ind_[int(base[j])]]
    print(f'{i}|{base}⟩ -> ', end='')
    for j in multi:
        print(f'|{j}⟩ ', end='')
        # print(f'{int(j, 2)}|{j}⟩ ', end='')
    print()

In [ ]:
# Qudit
3 = 3
nqd = 3 - 1
n = 2**nqd
for i in range(3):
    for j in range(3):
        a = f'a{str(i)}{str(j)}|{str(i)}⟩⟨{str(j)}|'
        print(a, end=' ')
    print()
print()
# Qubit
for i in range(n):
    for j in range(n):
        ii = bin(i)[2:].zfill(nqd)
        jj = bin(j)[2:].zfill(nqd)
        a = f'a{str(i)}{str(j)}|{ii}⟩⟨{jj}|'
        print(a, end=' ')
    print()

In [ ]:
# Qudit
3 = 4
nqd = 3 - 1
n = 2**nqd
print('\\begin{pmatrix}')
for i in range(3):
    for j in range(3):
        a = f'a_{{{str(i)}{str(j)}}}'
        if j < 3 - 1:
            print(a, end=' & ')
        else:
            print(a, end=' \\\\\n')
print('\\end{pmatrix}')
print()
# Qubit
ind = {}
for i in range(n):
    num1 = bin(i).count('1')
    ind[i] = [num1]
print(ind)
print('\\begin{pmatrix}')
for i in range(3):
    for j in range(3):
        a = f'a_{{{str(ind[i][0])}{str(ind[j][0])}}}'
        if j < 3 - 1:
            print(a, end=' & ')
        else:
            print(a, end=' \\\\\n')
print('\\end{pmatrix}')

In [ ]:
3 = 3
n = 2**(3 - 1)
a1 = unitary_group.rvs(3)
a2 = symmetric_encoding(a1)
print(a1)
print(a2)
p = np.eye(n) - symmetric_encoding(np.eye(3))
print(p)
a2 += p
is_unitary(a2), is_symmetric(a2)

In [ ]:
3 = 3
n = 2**(3 - 1)
i1 = np.eye(3)
i2 = symmetric_encoding(i1)
p = np.eye(n) - symmetric_encoding(np.eye(3))
print(i1)
print(i2)
print(p)
is_symmetric(i2), matrix_rank(i2)

In [ ]:
# U2⊗U2 Rxx Ryy Rzz SWAP preserve symmetry
3 = 3
a1 = unitary_group.rvs(3)
b1 = np.random.rand(3) + 1j * np.random.rand(3)
b1 /= norm(b1)
a2 = symmetric_encoding(a1)
b2 = symmetric_encoding(b1)
# print(a1)
# print(a2)
# print(b1)
# print(b2)
t = np.random.uniform(-np.pi, np.pi)
k = Circuit() + Rzz(t).on([0, 1])
# k = Circuit() + UN(UnivMathGate('', unitary_group.rvs(2)), 2)
display_svg(k.svg())
k = k.matrix()
# print(k)
# print(a2 @ k)
# print(k @ a2)
# print(k @ b2)
is_symmetric(a2 @ k), is_symmetric(k @ a2), is_symmetric(k @ b2)

In [ ]:
# How to restore a reduce density matrix
np.random.seed(42)
a = np.random.rand(2) + 1j * np.random.rand(2)
b = np.random.rand(2) + 1j * np.random.rand(2)
a /= norm(a)
b /= norm(b)
psi_ab = np.kron(a, b)
rho_a = np.outer(a, a.conj())
rho_b = np.outer(b, b.conj())
rho_ab = np.kron(rho_a, rho_b)
print(rho_a)
print(rho_b)
print(rho_ab)
print(psi_ab.conj() @ rho_ab @ psi_ab)
fidelity(rho_ab, psi_ab)